# Misbehavior Detection in Vehicular Networks with Federated Learning

## Imports

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import locale
import flwr as fl
import numpy as np
import pandas as pd
from abc import ABC
from metrics import *
import seaborn as sns
import tensorflow as tf
from itertools import cycle
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.utils import shuffle
from typing import Optional, Tuple, Dict, Any
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_recall_curve, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, auc
from sklearn.metrics import PrecisionRecallDisplay, RocCurveDisplay, roc_curve
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.set_visible_devices(physical_devices[1:], 'GPU')
  logical_devices = tf.config.list_logical_devices('GPU')
  assert len(logical_devices) == len(physical_devices) - 1
except:
  pass

tf.get_logger().setLevel('ERROR')
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
locale._override_localeconv = {'thousands_sep': '.'}

from IPython.display import display, HTML
display(HTML("<style>.container { width:65% !important; }</style>"))

2024-11-06 10:02:55.494745: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 10:02:55.508483: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 10:02:55.512413: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Support Functions

In [22]:
def load_veremi(csv_file: str, feature: str, label: str, delimiter=','):
    # Import VeReMi Dataset
    data = pd.read_csv(csv_file, delimiter=delimiter)

    data_normal = data.loc[data['attack_type'] == 0]
    data_atk = data.loc[data['attack_type'] != 0]
    atk_size = int(data_atk.shape[0])
    data = pd.concat([data_normal.sample(atk_size), data_atk])
    data = shuffle(data)

    dataset = data
    target = data[data.columns[-1:]]
    data = data[data.columns[0:-1]]

    # normalize data
    data = (data - data.mean()) / data.std()

    # Fill NaN values in specified columns with 0.0
    data[['noise_rcv_0', 'noise_rcv_1', 'spd_noise_rcv_0', 'spd_noise_rcv_1',
          'noise_snd_0', 'noise_snd_1', 'spd_noise_snd_0', 'spd_noise_snd_1']] = \
        data[['noise_rcv_0', 'noise_rcv_1', 'spd_noise_rcv_0', 'spd_noise_rcv_1',
              'noise_snd_0', 'noise_snd_1', 'spd_noise_snd_0', 'spd_noise_snd_1']].fillna(0.0)

    # label binarize one-hot style
    lb = preprocessing.LabelBinarizer()
    lb.fit(target)
    if label == 'multiclass':
        target = lb.transform(target)
    else:
        target = lb.transform(target)
        target = MultiLabelBinarizer().fit_transform(target)

    # Create training and test data
    train_data, test_data, train_labels, test_labels = train_test_split(
        data,
        target,
        train_size=Config.data_train_size,
        test_size=Config.data_test_size,
        # random_state=42
    )

    return train_data, test_data, train_labels, test_labels, lb, dataset

## Config Class

In [35]:
class Config:
    csv = "./VeReMi.csv"
    model_type = "mlp"
    label = "multiclass"
    feature = "feat4"
    batch_size = 128
    epochs = 5
    rounds = 80
    learning_rate = 3e-4 #1e-3
    min_available_clients = 2
    fraction_fit = 0.1
    early_stop_patience = 3
    early_stop_monitor = "loss"
    early_stop_min_delta = 1e-4
    early_stop_restore_best_weights = True
    data_train_size = 0.8
    data_test_size = 0.2
    output_path = f"results/{feature}/{label}/"
    performance_file = "performance.csv"
    weights_file = "model_weights.npz"

## Metrics

In [36]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall_keras = true_positives / (possible_positives + K.epsilon())
    return recall_keras


def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision_keras = true_positives / (predicted_positives + K.epsilon())
    return precision_keras


def f1(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + K.epsilon()))

## VeReMi Base Class

In [58]:
class VeremiBase(ABC):
    def __init__(self, data_file: str, model_type: str, label: str, feature: str, activation: str = "softmax"):
        """ The Veremi Client Constructor
            :param model_type: Keras Model Type ('mlp' or 'lstm'
            :param label: Model label type ('binary', 'multiclass', 'atk_1', 'atk_2', 'atk_4', 'atk_8', 'atk_16')
            :param feature: Feature to evaluate ('feat1', 'feat2', 'feat3')
        """
        self.lb = None
        self.dataset = None
        self.train_data = None
        self.test_data = None
        self.train_labels = None
        self.test_labels = None
        self.model = None
        self.data_file = data_file
        self.label = label
        self.feature = feature
        self.model_type = model_type
        self.activation = activation

        self.load_veremi()
        self.create_model()

    def create_model(self):
        layer1, layer2, layer3, layer4, layer5, layer6, layer7, layer8, layer9, output = \
            None, None, None, None, None, None, None, None, None, None
        name = self.label + "-" + self.model_type + "-" + self.feature
        if self.model_type == 'mlp':
            self.model = tf.keras.models.Sequential([
                keras.layers.Input(shape=(self.train_data.shape[1],)),
                keras.layers.Dense(48, activation="relu"),
                keras.layers.Dropout(0.5),
                keras.layers.Dense(24, activation="relu"),
                # keras.layers.Dense(256, activation="relu"),
                keras.layers.Dropout(0.5),
                # keras.layers.Dense(128, activation="relu"),
                # keras.layers.Dropout(0.25),
                keras.layers.Dense(self.train_labels.shape[1], activation=self.activation)
            ], name=name)
        else:
            pass

        # ML Model
        # self.model = keras.Model(inputs=layer1, outputs=output, name=name)
        self.model.compile(
            loss=keras.losses.BinaryCrossentropy(),
            optimizer=keras.optimizers.Nadam(learning_rate=Config.learning_rate),
            metrics=[f1]
        )
        self.model.summary()

    def load_veremi(self):
        print("Loading dataset in " + self.__class__.__name__ + "...")
        self.train_data, self.test_data, self.train_labels, self.test_labels, self.lb, self.dataset = load_veremi(
            self.data_file,
            feature=self.feature,
            label=self.label
        )

## VeReMi Client Class

In [59]:
class VeremiClient(VeremiBase, fl.client.NumPyClient):
    def __init__(self, data_file: str, model_type: str, label: str, feature: str):
        VeremiBase.__init__(self, data_file, model_type, label, feature)
        self.history = None

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config, verbose):
        print("Training...")
        early_stopping = keras.callbacks.EarlyStopping(
            monitor=Config.early_stop_monitor,
            patience=Config.early_stop_patience,
            min_delta=Config.early_stop_min_delta,
            restore_best_weights=Config.early_stop_restore_best_weights
        )
        if parameters is not None:
            self.model.set_weights(parameters)
        self.history = self.model.fit(
            self.train_data,
            self.train_labels,
            batch_size=config["batch_size"],
            epochs=config["epochs"],
            # callbacks=[early_stopping],
            validation_data=(self.test_data, self.test_labels),
            verbose=verbose,
        )
        result = {
            "f1_score:": float(self.history.history['f1'][-1]),
            "f1_val": float(self.history.history['val_f1'][-1]),
        }
        print("Finished!")
        return self.model.get_weights(), len(self.train_data), result

### Create Client

In [67]:
client = VeremiClient(Config.csv, Config.model_type, Config.label, Config.feature)

Loading dataset in VeremiClient...


Model: "multiclass-mlp-feat4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_27 (Dense)                │ (None, 48)             │           912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 24)             │         1,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 24)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │            25 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,113 (8.25 KB)

 Trainable params: 2,113 (8.25 KB)

 Non-trainable params: 0 (0.00 B)

## VeReMi DataSet

In [61]:
client.dataset

,receiver,rcvTime,pos_rcv_0,pos_rcv_1,noise_rcv_0,noise_rcv_1,spd_rcv_0,spd_rcv_1,spd_noise_rcv_0,spd_noise_rcv_1,pos_snd_0,pos_snd_1,noise_snd_0,noise_snd_1,spd_snd_0,spd_snd_1,spd_noise_snd_0,spd_noise_snd_1,attack_type
807,11,10829.068186,3601.852273,5525.183273,0.0,0.0,-2.473960,30.200530,0.0,0.0,3630.236601,5338.558205,0.0,0.0,3.977073,-32.110010,0.0,0.0,1
2479,28,10883.171586,3591.859940,5730.385685,0.0,0.0,1.377828,-32.055592,0.0,0.0,3594.464795,5754.969901,0.0,0.0,-0.016000,-27.461340,0.0,0.0,0
5972,7,10803.137592,3615.247057,5432.746930,0.0,0.0,3.684407,-29.754297,0.0,0.0,3590.580075,5582.064172,0.0,0.0,-2.795552,34.125454,0.0,0.0,1
3969,10,10841.103264,3589.435682,5926.820363,0.0,0.0,1.273319,29.906840,0.0,0.0,3587.819434,5888.859048,0.0,0.0,1.041610,29.930948,0.0,0.0,0
4626,15,10814.137587,3598.518840,6010.670246,0.0,0.0,-3.420759,-29.683266,0.0,0.0,3585.097268,5958.836102,0.0,0.0,2.537669,34.177445,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774,11,10818.068187,3644.242959,5189.792013,0.0,0.0,-4.858861,31.148183,0.0,0.0,3596.784533,5692.610069,0.0,0.0,1.387058,-32.299526,0.0,0.0,1
3574,3,10808.068190,3612.791099,5518.096011,0.0,0.0,2.669311,-31.705822,0.0,0.0,3602.411691,6015.606367,0.0,0.0,-3.692996,-32.064974,0.0,0.0,1
5955,7,10801.137587,3608.342239,5492.341738,0.0,0.0,2.512103,-29.842618,0.0,0.0,3596.177794,5513.732534,0.0,0.0,-2.799638,34.175327,0.0,0.0,1
1880,13,10849.685935,3616.335810,5477.612937,0.0,0.0,3.287230,-29.433561,0.0,0.0,3597.764030,5618.005943,0.0,0.0,2.528556,-30.038074,0.0,0.0,0


In [25]:
client.train_data.describe()

,receiver,rcvTime,pos_rcv_0,pos_rcv_1,noise_rcv_0,noise_rcv_1,spd_rcv_0,spd_rcv_1,spd_noise_rcv_0,spd_noise_rcv_1,pos_snd_0,pos_snd_1,noise_snd_0,noise_snd_1,spd_snd_0,spd_snd_1,spd_noise_snd_0,spd_noise_snd_1
count,300.000000,300.000000,300.000000,300.000000,300.0,300.0,300.000000,300.000000,300.0,300.0,300.000000,300.000000,300.0,300.0,300.000000,300.000000,300.0,300.0
mean,-0.058477,-0.062413,-0.021971,0.036123,0.0,0.0,-0.003393,0.017363,0.0,0.0,-0.028005,0.059644,0.0,0.0,0.067950,-0.040207,0.0,0.0
std,0.970294,0.959303,0.959521,1.003226,0.0,0.0,0.974592,1.002175,0.0,0.0,0.949940,0.988438,0.0,0.0,0.952851,0.999288,0.0,0.0
min,-1.385531,-1.132289,-0.400171,-1.526945,0.0,0.0,-1.778743,-1.273079,0.0,0.0,-0.364958,-1.746250,0.0,0.0,-3.737821,-1.388057,0.0,0.0
25%,-0.849564,-0.813016,-0.353934,-0.930578,0.0,0.0,-0.836232,-0.883794,0.0,0.0,-0.316098,-0.574218,0.0,0.0,-0.658161,-1.066766,0.0,0.0
50%,-0.206404,-0.388595,-0.310585,-0.037007,0.0,0.0,-0.153370,-0.159418,0.0,0.0,-0.284331,0.055354,0.0,0.0,-0.011457,-0.067229,0.0,0.0
75%,0.463554,0.482538,-0.194109,0.950695,0.0,0.0,0.796462,1.075685,0.0,0.0,-0.208193,0.868647,0.0,0.0,0.813496,0.997676,0.0,0.0
max,2.580622,2.380555,4.056364,1.790107,0.0,0.0,3.840487,1.417343,0.0,0.0,4.074901,1.951789,0.0,0.0,3.254619,1.218504,0.0,0.0


In [63]:
attack = len(client.dataset[client.dataset.attack_type != 0])
normal = len(client.dataset[client.dataset.attack_type == 0])
total = attack + normal
print('Attackers:\n    Total: {:,d}\n    Attack: {:,d} ({:.2f}% of total)\n'.format(
    total, attack, 100 * attack / total))

Attackers:
    Total: 376
    Attack: 188 (50.00% of total)



## Fit the MLP Model

### Load Model Params

In [68]:
initial_parameters = None
file = Config.output_path + Config.weights_file
if os.path.exists(file):
    npzfile = np.load(file)
    params = [npzfile[x] for x in npzfile]
    params = fl.common.ndarrays_to_parameters(params)
    initial_parameters = fl.common.parameters_to_ndarrays(params)
    print("Setting model params...")
    client.model.set_weights(initial_parameters)

In [69]:
results = client.fit(
    parameters=None, # initial_parameters,
    config={
        "batch_size": Config.batch_size,
        "epochs": Config.epochs
    },
    verbose=1
)

Training...
Epoch 1/5


/home/hlazuroz/.local/share/virtualenvs/ic-veremi-fed-learning-Xdbt6niO/lib/python3.10/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


AttributeError: module 'keras.api.backend' has no attribute 'sum'

In [ ]:
train_f1_score = client.history.history['f1']
test_f1_score = client.history.history['val_f1']
train_loss = client.history.history['loss']
test_loss = client.history.history['val_loss']
epochs = range(1, Config.epochs + 1)
loss, num_examples, metrics = results

fig, (ax1, ax2) = plt.subplots(1,2)
fig.suptitle('Model Performance')
fig.set_figwidth(15)
fig.set_figheight(5)

ax1.plot(epochs, train_loss, '-g', label="Train Loss")
ax1.plot(epochs, test_loss, '-b', label="Test Loss")
ax1.legend()
ax1.set(xlabel='Epochs', ylabel='Loss')
ax1.set_title('Loss')

ax2.plot(epochs, train_f1_score, '-g', label="Train F1 Score")
ax2.plot(epochs, test_f1_score, '-b', label="Test F1 Score")
ax2.legend()
ax2.set(xlabel='Epochs', ylabel='F1 Score')
ax2.set_title('F1-Score')

plt.show()

In [ ]:
def plot_pr_roc_curves(probabilities: Any):
    n_classes = client.test_labels.shape[1]
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    precision = dict()
    recall = dict()
    pr_auc = dict()
        
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(client.test_labels[:, i], probabilities[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

        precision[i], recall[i], _ = precision_recall_curve(client.test_labels[:, i], probabilities[:, i])
        pr_auc[i] = auc(recall[i], precision[i])

    # First aggregate all false positive rates
    all_precision = np.unique(np.concatenate([precision[i] for i in range(n_classes)]))
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    mean_recall = np.zeros_like(all_precision)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
        mean_recall += np.interp(all_precision, precision[i], recall[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes
    mean_recall /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    
    precision["macro"] = all_precision
    recall["macro"] = mean_recall
    pr_auc["macro"] = auc(recall["macro"], precision["macro"])

    lw = 2
    
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.suptitle('PR and ROC Curves - Multiclass')
    fig.set_figwidth(15)
    fig.set_figheight(5)

    colors = cycle(["b", "g", "r", "c", "m", "y"])
    
    # ROC Curve
    ax1.plot(
        fpr["macro"],
        tpr["macro"],
        label="Macro Avg (area = {0:0.2f})".format(roc_auc["macro"]),
        color="navy",
        linestyle=":",
        linewidth=4,
        alpha=0.5,        
    )
    for i, color in zip(range(n_classes), colors):
        label_classes = int(client.lb.classes_[i])
        ax1.plot(
            fpr[i],
            tpr[i],
            color=color,
            lw=lw,
            label="Class {0} (area = {1:0.2f})".format(label_classes, roc_auc[i]),
            alpha=0.5
        )
    ax1.set_title(f"ROC Curve - {Config.feature} - {Config.label}")
    ax1.set_xlabel("False Positive Rate")
    ax1.set_ylabel("True Positive Rate")
    ax1.legend()

    # PR curve
    ax2.plot(
        precision["macro"],
        recall["macro"],
        label="Macro Avg (area = {0:0.2f})".format(pr_auc["macro"]),
        color="navy",
        linestyle=":",
        linewidth=4,
        alpha=0.5,
    )
    for i, color in zip(range(n_classes), colors):
        label_classes = int(client.lb.classes_[i])
        ax2.plot(
            precision[i],
            recall[i],
            color=color,
            lw=lw,
            label="Class {0} (area = {1:0.2f})".format(label_classes, pr_auc[i]),
            alpha=0.5
        )
    ax2.set_title(f"PR Curve - {Config.feature} - {Config.label}")    
    ax2.set_xlabel("False Positive Rate")
    ax2.set_ylabel("True Positive Rate")
    ax2.legend()
    
    plt.show()

In [ ]:
probabilities = client.model.predict(client.test_data)
inverse_target = client.lb.inverse_transform(client.test_labels)
prediction = None

if Config.label == 'multiclass':
    prediction = client.lb.inverse_transform(probabilities)
    # TODO: PLOT MULTICLASS 
    plot_pr_roc_curves(probabilities)
else:
    pos_label = 1 if Config.label == 'binary' else int(Config.label.split("_")[1])
    # Best threshold
    precision, recall, thresholds = precision_recall_curve(
        inverse_target,
        probabilities[:, 1],
        pos_label=pos_label
    )
    # convert to f score
    np.seterr(divide='ignore', invalid='ignore')
    fscore = (2 * precision * recall) / (precision + recall)
    np.nan_to_num(fscore, copy=False)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    print('Best Threshold=%f, F-Score=%.3f' % (thresholds[ix], fscore[ix]))
    print("-" * 70)
    
    prediction = np.where(np.array(probabilities[:, 1]) >= thresholds[ix], pos_label, 0)
    
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.suptitle('PR and ROC Curves')
    fig.set_figwidth(15)
    fig.set_figheight(5)

    # PR Curve
    PrecisionRecallDisplay.from_predictions(inverse_target, probabilities[:, 1], pos_label=pos_label, ax=ax1)
    no_skill = len(inverse_target[inverse_target == 1]) / len(inverse_target)
    ax1.plot([0, 1], [no_skill, no_skill], linestyle='--', color="grey", label='No Skill')
    ax1.scatter(recall[ix], precision[ix], marker='o', color='black', label='Best threshold')
    ax1.set_title(f"PR Curve - {Config.feature} - {Config.label}")
    ax1.legend()

    # ROC curve
    RocCurveDisplay.from_predictions(inverse_target, probabilities[:, 1], pos_label=pos_label, ax=ax2)
    ax2.plot([0, 1], [0, 1], color="grey", lw=1, linestyle="--")
    ax2.set_title(f"ROC Curve - {Config.feature} - {Config.label}")
    
    plt.show()

### Classification Report

In [ ]:
classlist = []
for cl in client.lb.classes_:
    classlist.append('class ' + str(int(cl)))

print(classification_report(inverse_target, prediction, target_names=classlist, digits=3, zero_division=0))
print("-" * 70)

### Confusion Matrix

In [ ]:
cm = confusion_matrix(inverse_target, prediction, labels=client.lb.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=client.lb.classes_)
disp.plot()
plt.title(f"Confusion Matrix - {Config.feature} - {Config.label}")
plt.show()

### Performance

In [ ]:
name = Config.label + "-" + Config.model_type + "-" + Config.feature
if Config.label == 'multiclass':
    prscore = precision_score(inverse_target, prediction, average='macro', zero_division=0)
    rcscore = recall_score(inverse_target, prediction, average='macro', zero_division=0)
    f1score = f1_score(inverse_target, prediction, average='macro', zero_division=0)
    accscore = accuracy_score(inverse_target, prediction)
else:
    prscore = precision_score(inverse_target, prediction, pos_label=pos_label, zero_division=0)
    rcscore = recall_score(inverse_target, prediction, pos_label=pos_label, zero_division=0)
    f1score = f1_score(inverse_target, prediction, pos_label=pos_label, zero_division=0)
    accscore = accuracy_score(inverse_target, prediction)
data_performance = {name: [prscore, rcscore, f1score, accscore]}
df_performance = pd.DataFrame.from_dict(data_performance, orient='index', columns=["precision", "recall", "f1score", "accuracy"])
df_performance